# Applied Data science Capstone: The Battle of Neighborhoods

## "Best Neighborhood in Amsterdam"

## 1. Introduction : Business Problem

Starting a new business need more research to come up with the best idea in the best place. The aim of this project is to find for stakeholders **which is the most promissing business in which the perfect neighborhoods** to start, especially in a city with high density like **Amsterdam**. 

There are lots of business projects like **Restaurant, Grocery, Clothes Store**,… scattered between neighborhood in **Amsterdam**. It would be interesting if we have some groups with close similarities to define which is the most common  business in area. Also, it is important to compare with **number of population in each neighbohood** to have more idea about suitable business.

Therefore, we will use data science to generate a few propositions of neighborhoods based on this criteria, in order to find suitable business kind in best location for stakeholders.

## 2. Data section

To answer the main question in the previous section, we should define :

    *Most common business kinds (Restaurants, Groceries, Stores, Coffees,…) in each neighborhood 
    *number of total population in neighborhood


First, we need to download the list of all neighborhoods in Amsterdam (Wijk) with demographic informations to extract number of habitants, from website https://claircitydata.cbs.nl/

Then, we will get all venues located in the corresponding neighborhoods using using Foursquare API, in order to classify the most common business in each neighborhood

### Neighborhoods in Amsterdam-Wijk

First we import data about neighborhoods in Amsterdam and their demographic informations

In [5]:
import pandas as pd
import numpy as np

In [64]:
df=pd.read_csv("C:/Users/pc/Desktop/Coursera/Final Assignment_Capstone project/wijken_buurten.csv",delimiter=",")

In [65]:
df.head()

,subject,region_name,regio_type,region_code,ninhabitants,nmen,nwomen,nage_0_to_15,nage_15_to_25,nage_25_to_45,nage_45_to_65,nage_65_older,nunmarried,nmarried,ndivorced,nwidowed,nimmigrant_western,nimmigrant_nonwestern,nimmigrant_marokko,nimmigrant_antiles_aruba,nimmigrant_surinam,nimmigrant_turkey,nimmigrant_other_non_western,nhouseholds,nhh_single_person,nhh_no_children,nhh_with_children,ave_househ_size,populatio_density,area_total,area_land,area_water,urbanisation_grade,address_density,geojson,lon,lat
0,Amsterdam,Amsterdam,Gemeente,GM0363,833624,411608,422016,127224,108370,293154,204741,100135,531112,203974,73962,24576,140901,289823,75065,12227,66119,42622,93790,456462,251828,92709,111925,1.8,5042.0,21949,16533,5416,1.0,6011.0,"{""type"": ""FeatureCollection"",""crs"": { ""type"": ...",4.879568,52.381000
1,Burgwallen-Oude Zijde,Amsterdam,Wijk,WK036300,4280,2340,1935,255,675,2030,980,355,3280,660,290,65,1275,640,25,40,80,40,455,3070,2160,655,265,1.4,12255.0,40,35,6,1.0,7619.0,"{""type"": ""FeatureCollection"",""crs"": { ""type"": ...",4.896943,52.372566
2,Kop Zeedijk,Amsterdam,Buurt,BU03630000,1020,570,445,50,140,520,260,60,765,175,70,15,320,145,5,10,20,10,105,740,520,170,55,1.4,16818.0,8,6,2,1.0,6649.0,"{""type"": ""FeatureCollection"",""crs"": { ""type"": ...",4.900280,52.375937
3,Oude Kerk e.o.,Amsterdam,Buurt,BU03630001,670,365,300,30,130,320,145,55,540,70,55,15,205,105,5,10,15,10,70,500,370,95,45,1.3,7615.0,9,9,0,1.0,7361.0,"{""type"": ""FeatureCollection"",""crs"": { ""type"": ...",4.896156,52.373793
4,Burgwallen Oost,Amsterdam,Buurt,BU03630002,1610,880,730,120,250,755,340,155,1230,255,115,25,495,275,10,15,35,10,210,1120,770,260,105,1.4,25867.0,7,6,1,1.0,7544.0,"{""type"": ""FeatureCollection"",""crs"": { ""type"": ...",4.898757,52.372815


In [73]:
df=df.drop(df.index[0])

In [77]:
amster = df[df['regio_type'] == 'Wijk'].reset_index(drop=True)
amster.head()

,subject,region_name,regio_type,region_code,ninhabitants,nmen,nwomen,nage_0_to_15,nage_15_to_25,nage_25_to_45,nage_45_to_65,nage_65_older,nunmarried,nmarried,ndivorced,nwidowed,nimmigrant_western,nimmigrant_nonwestern,nimmigrant_marokko,nimmigrant_antiles_aruba,nimmigrant_surinam,nimmigrant_turkey,nimmigrant_other_non_western,nhouseholds,nhh_single_person,nhh_no_children,nhh_with_children,ave_househ_size,populatio_density,area_total,area_land,area_water,urbanisation_grade,address_density,geojson,lon,lat
0,Burgwallen-Oude Zijde,Amsterdam,Wijk,WK036300,4280,2340,1935,255,675,2030,980,355,3280,660,290,65,1275,640,25,40,80,40,455,3070,2160,655,265,1.4,12255.0,40,35,6,1.0,7619.0,"{""type"": ""FeatureCollection"",""crs"": { ""type"": ...",4.896943,52.372566
1,Burgwallen-Nieuwe Zijde,Amsterdam,Wijk,WK036301,3960,2150,1805,220,595,2100,775,285,3140,545,245,35,1255,630,35,35,65,40,465,2900,2105,625,175,1.4,6935.0,73,57,16,1.0,9206.0,"{""type"": ""FeatureCollection"",""crs"": { ""type"": ...",4.895518,52.375703
2,Grachtengordel-West,Amsterdam,Wijk,WK036302,6475,3415,3060,585,840,2215,1820,1030,4195,1725,420,150,2050,610,10,45,70,45,445,4200,2660,1030,520,1.5,14469.0,57,45,12,1.0,10577.0,"{""type"": ""FeatureCollection"",""crs"": { ""type"": ...",4.886836,52.373709
3,Grachtengordel-Zuid,Amsterdam,Wijk,WK036303,5440,2960,2480,535,680,2095,1360,775,3605,1370,350,120,1550,625,20,35,70,70,435,3505,2260,785,470,1.5,10476.0,64,52,12,1.0,9136.0,"{""type"": ""FeatureCollection"",""crs"": { ""type"": ...",4.893349,52.364821
4,Nieuwmarkt/Lastage,Amsterdam,Wijk,WK036304,9605,4990,4615,810,1200,3200,2960,1450,6380,2080,895,260,2490,1280,85,80,265,75,780,6325,4105,1360,865,1.5,13390.0,107,72,35,1.0,7025.0,"{""type"": ""FeatureCollection"",""crs"": { ""type"": ...",4.905641,52.372436


In [78]:
amster.shape

(99, 37)

In [79]:
amster=amster[["subject","lon","lat","ninhabitants"]]

In [83]:
amster.head()

,subject,lon,lat,ninhabitants
0,Burgwallen-Oude Zijde,4.896943,52.372566,4280
1,Burgwallen-Nieuwe Zijde,4.895518,52.375703,3960
2,Grachtengordel-West,4.886836,52.373709,6475
3,Grachtengordel-Zuid,4.893349,52.364821,5440
4,Nieuwmarkt/Lastage,4.905641,52.372436,9605


In [81]:
amster_data=amster.rename({"subject":"Neighborhood","ninhabitants":"habitants"},axis=1)

In [82]:
amster_data.head()

,Neighborhood,lon,lat,habitants
0,Burgwallen-Oude Zijde,4.896943,52.372566,4280
1,Burgwallen-Nieuwe Zijde,4.895518,52.375703,3960
2,Grachtengordel-West,4.886836,52.373709,6475
3,Grachtengordel-Zuid,4.893349,52.364821,5440
4,Nieuwmarkt/Lastage,4.905641,52.372436,9605


In [24]:
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium

In [86]:
address = 'AMSTERDAM,AM'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Amsterdam are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Amsterdam are 52.3116337, 4.9471091.


In [88]:
# create map of Amsterdam using latitude and longitude values
map_Amsterdam= folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lon in zip(amster_data['lat'], amster_data['lon']):
    
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Amsterdam)  
    
map_Amsterdam

Then, we use Fosrquare API to get data about venues in Wijk

In [33]:
CLIENT_ID = 'NCAIFWZAXGOLN1R1IVF4C3JUP1CK42TU2440CAJKST1OVXKJ' #  Foursquare ID
CLIENT_SECRET = 'AHNILBWUGPXLVC0UQ0YRXB1VF2XZBISWY0SHTVZ2BXSOK55P' #  Foursquare Secret
VERSION = '20201219' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: NCAIFWZAXGOLN1R1IVF4C3JUP1CK42TU2440CAJKST1OVXKJ
CLIENT_SECRET:AHNILBWUGPXLVC0UQ0YRXB1VF2XZBISWY0SHTVZ2BXSOK55P


In [89]:
neighborhood_latitude = amster_data.loc[0, 'lat'] # neighborhood latitude value
neighborhood_longitude = amster_data.loc[0, 'lon'] # neighborhood longitude value

neighborhood_name = amster_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Burgwallen-Oude Zijde are 52.3725658613947, 4.8969427210672904.


In [44]:
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [90]:
# type your answer here
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=NCAIFWZAXGOLN1R1IVF4C3JUP1CK42TU2440CAJKST1OVXKJ&client_secret=AHNILBWUGPXLVC0UQ0YRXB1VF2XZBISWY0SHTVZ2BXSOK55P&v=20201219&ll=52.3725658613947,4.8969427210672904&radius=500&limit=100'

In [91]:
results = requests.get(url).json()

In [93]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [94]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

<ipython-input-94-561c05f0fdd1>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,Rosalia's Menagerie,Cocktail Bar,52.371678,4.899174
1,De Koffieschenkerij,Coffee Shop,52.374043,4.898427
2,Sofitel Legend The Grand Amsterdam,Hotel,52.371093,4.895410
3,Wynand Fockink,Liquor Store,52.372301,4.895253
4,De Bijenkorf,Department Store,52.373753,4.895271


In [95]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


In [96]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


In [ ]:
amsterdam_venues = getNearbyVenues(names=amster_data['Neighborhood'],
                                   latitudes=amster_data['lat'],
                                   longitudes=amster_data['lon']
                                  )

In [98]:
print('There are {} uniques categories.'.format(len(amsterdam_venues['Venue Category'].unique())))

There are 293 uniques categories.


In [113]:
amsterdam_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Burgwallen-Oude Zijde,52.372566,4.896943,Rosalia's Menagerie,52.371678,4.899174,Cocktail Bar
1,Burgwallen-Oude Zijde,52.372566,4.896943,De Koffieschenkerij,52.374043,4.898427,Coffee Shop
2,Burgwallen-Oude Zijde,52.372566,4.896943,Sofitel Legend The Grand Amsterdam,52.371093,4.895410,Hotel
3,Burgwallen-Oude Zijde,52.372566,4.896943,Wynand Fockink,52.372301,4.895253,Liquor Store
4,Burgwallen-Oude Zijde,52.372566,4.896943,De Bijenkorf,52.373753,4.895271,Department Store


Great! Now that we have our data about city of Amsterdam, and about all venues in Wijk, let s explore our data and anlyze it

The following section will explain the methodology to work with in order to solve this project

## 3 . Methodology section

In this project, our aim to define a few propositions of neighborhoods candidates with their profils and match them with their demographic information such as number of habitants. 

First, We suggest to use the explore function to get the most common venue categories in each neighborhood, and then use this feature to group the neighborhoods into clusters. 

Then, we  will use the **k-means clustering algorithm** to complete this analysis, and to have similar neighborhoods based on their profils. Finally, we get our clusters sorted by **number of population** in each neighborhood and then the global idea about the suitable business in suitable neighborhood.

####  Most common venues by Neighborhoods

In [ ]:
amsterdam_venues['Venue Category'].value_counts

In [116]:
# one hot encoding
amsterdam_onehot = pd.get_dummies(amsterdam_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
amsterdam_onehot['Neighborhood'] = amsterdam_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [amsterdam_onehot.columns[-1]] + list(amsterdam_onehot.columns[:-1])
amsterdam_onehot = amsterdam_onehot[fixed_columns]

amsterdam_onehot.head()

,Zoo Exhibit,Accessories Store,Afghan Restaurant,African Restaurant,American Restaurant,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bar,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bike Shop,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brasserie,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Buffet,Building,Burger Joint,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Camera Store,Campground,Canal,Candy Store,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cruise Ship,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dutch Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Fondue Restaurant,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Forest,Fountain,French Restaurant,Fried Chicken Joint,Friterie,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Herbs & Spices Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Lebanese Restaurant,Library,Liquor Store,Lounge,Marijuana Dispensary,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,Nightclub,Noodle House,North Indian Restaurant,Office,Optical Shop,Organic Grocery,Outdoor Sculpture,Outdoors & Recreation,Palace,Paper / Office Supplies Store,Park,Parking,Pastry Shop,Performing Arts Venue,Peruvian Restaurant,Pet Café,Pet Store,Pharmacy,Piano Bar,Pier,Pilates Studio,Pizza Place,Planetarium,Playground,Plaza,Pool,Pool Hall,Pop-Up Shop,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Rest Area,Restaurant,Road,Rock Club,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Seafood Restaurant,Shoe Repair,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Speakeasy,Spiritual Center,Sporting Goods Shop,Sports Club,Squash Court,Stables,Stadium,Steakhouse,Storage Facility,Supermarket,Sushi Restaurant,Tanning Salon,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Tram Station,Tunnel,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Video Game 

In [117]:
amsterdam_onehot.shape

(3202, 293)

In [ ]:
amsterdam_grouped = amsterdam_onehot.groupby('Neighborhood').mean().reset_index()

In [120]:
num_top_venues = 5

for hood in amsterdam_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = amsterdam_grouped[amsterdam_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Amstel III/Bullewijk----
                    venue  freq
0  Furniture / Home Store  0.19
1       Electronics Store  0.12
2              Restaurant  0.12
3                Building  0.06
4     Rental Car Location  0.06


----Apollobuurt----
            venue  freq
0           Hotel  0.16
1     Bridal Shop  0.05
2       Juice Bar  0.05
3            Café  0.05
4  Breakfast Spot  0.05


----Banne Buiksloot----
                venue  freq
0            Bus Stop  0.22
1                Park  0.22
2  Turkish Restaurant  0.11
3       Shopping Mall  0.11
4          Restaurant  0.11


----Bedrijventerrein Sloterdijk----
                 venue  freq
0      Motorcycle Shop   0.4
1      Automotive Shop   0.2
2        Auto Workshop   0.2
3  Rental Car Location   0.2
4          Zoo Exhibit   0.0


----Betondorp----
             venue  freq
0              Bar  0.50
1           Bakery  0.17
2           Market  0.17
3  Organic Grocery  0.00
4     Optical Shop  0.00


----Bijlmer Centrum (D,F,H)----
   

In [125]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [200]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = amsterdam_grouped['Neighborhood']

for ind in np.arange(amsterdam_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(amsterdam_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Amstel III/Bullewijk,Furniture / Home Store,Electronics Store,Restaurant,Gym,Fried Chicken Joint,Rental Car Location,Bistro,Asian Restaurant,Intersection,Coffee Shop
1,Apollobuurt,Hotel,Health Food Store,Baby Store,Café,Italian Restaurant,Bistro,Comedy Club,Steakhouse,Arts & Crafts Store,Bus Stop
2,Banne Buiksloot,Park,Bus Stop,Restaurant,Supermarket,Shopping Mall,Turkish Restaurant,Bakery,Zoo,Ethiopian Restaurant,Dutch Restaurant
3,Bedrijventerrein Sloterdijk,Motorcycle Shop,Automotive Shop,Auto Workshop,Rental Car Location,Zoo,Event Space,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant
4,Betondorp,Bar,Bakery,Market,Zoo,Falafel Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit


### K-Means Clustering

In [137]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

In [139]:
amsterdam_grouped_clustering = amsterdam_grouped.drop('Neighborhood', 1)

In [201]:
# set number of clusters
kclusters = 5

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(amsterdam_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 2, 0, 4, 0, 0, 0, 0, 0])

Now we merge result to our neighborhood data

In [202]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

amsterdam_merged = amster_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
amsterdam_merged = amsterdam_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

In [203]:
amsterdam_merged.head()

,Neighborhood,lon,lat,habitants,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Burgwallen-Oude Zijde,4.896943,52.372566,4280,0.0,Hotel,Bar,Coffee Shop,French Restaurant,Cheese Shop,Sandwich Place,Clothing Store,Marijuana Dispensary,Thai Restaurant,Breakfast Spot
1,Burgwallen-Nieuwe Zijde,4.895518,52.375703,3960,0.0,Bar,Hotel,Coffee Shop,French Restaurant,Marijuana Dispensary,Café,Restaurant,Cheese Shop,Pub,Church
2,Grachtengordel-West,4.886836,52.373709,6475,0.0,Hotel,Bar,Café,French Restaurant,Coffee Shop,Marijuana Dispensary,Sandwich Place,Art Museum,Art Gallery,Clothing Store
3,Grachtengordel-Zuid,4.893349,52.364821,5440,0.0,Hotel,Coffee Shop,Sandwich Place,Restaurant,Italian Restaurant,Café,Marijuana Dispensary,Multiplex,Plaza,Clothing Store
4,Nieuwmarkt/Lastage,4.905641,52.372436,9605,0.0,Bar,Hotel,Hostel,Coffee Shop,Grocery Store,Bed & Breakfast,Cocktail Bar,Restaurant,Pizza Place,Bike Rental / Bike Share


In [204]:
amsterdam_merged=amsterdam_merged.dropna(axis='index',how='any')

In [157]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [205]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(amsterdam_merged['lat'], amsterdam_merged['lon'], amsterdam_merged['Neighborhood'], amsterdam_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(int(cluster)), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Exploring  all Clusters

In [220]:
first=amsterdam_merged.loc[amsterdam_merged['Cluster Labels'] == 0]
first.sort_values(by="habitants",ascending=False).head()

,Neighborhood,lon,lat,habitants,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
94,"Bijlmer Oost (E,G,K)",4.977314,52.320278,27190,0.0,Metro Station,Shopping Mall,Supermarket,Drugstore,Discount Store,Fast Food Restaurant,South American Restaurant,Middle Eastern Restaurant,Bus Stop,Bar
93,"Bijlmer Centrum (D,F,H)",4.953987,52.315127,23910,0.0,Clothing Store,Chinese Restaurant,Bakery,Italian Restaurant,Fast Food Restaurant,Supermarket,Sandwich Place,Restaurant,Plaza,Park
6,Jordaan,4.880786,52.374805,19390,0.0,Hotel,Bar,Italian Restaurant,Furniture / Home Store,Yoga Studio,Café,Indian Restaurant,Chocolate Shop,Coffee Shop,Restaurant
37,Landlust,4.854571,52.380731,19050,0.0,Tram Station,Coffee Shop,Vegetarian / Vegan Restaurant,Snack Place,Café,Playground,Bakery,Bar,Indonesian Restaurant,Flower Shop
33,Oostelijk Havengebied,4.939367,52.375589,18475,0.0,Café,Furniture / Home Store,Coffee Shop,Chinese Restaurant,Bus Stop,Supermarket,Bookstore,Spanish Restaurant,Fish Market,Shopping Mall


In [213]:
second=amsterdam_merged.loc[amsterdam_merged['Cluster Labels'] == 1]
second.sort_values(by="habitants",ascending=False)

,Neighborhood,lon,lat,habitants,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
66,Oostzanerwerf,4.876877,52.421728,8755,1.0,Soccer Field,Creperie,Pier,Bus Stop,Exhibit,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space
79,Eendracht,4.778767,52.379161,2385,1.0,Garden,Soccer Field,Event Space,Cafeteria,Zoo,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Exhibit


In [221]:
third=amsterdam_merged.loc[amsterdam_merged['Cluster Labels'] == 2]
third.sort_values(by="habitants",ascending=False).head()

,Neighborhood,lon,lat,habitants,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
78,Geuzenveld,4.801950,52.378998,15455,2.0,Bus Stop,Supermarket,Tram Station,Turkish Restaurant,Snack Place,Italian Restaurant,Drugstore,Market,Smoke Shop,Plaza
70,Banne Buiksloot,4.917565,52.407294,14175,2.0,Park,Bus Stop,Restaurant,Supermarket,Shopping Mall,Turkish Restaurant,Bakery,Zoo,Ethiopian Restaurant,Dutch Restaurant
88,Sloter-/Riekerpolder,4.816744,52.339611,13965,2.0,Bus Stop,Hotel,Café,Diner,Zoo,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit
68,Waterlandpleinbuurt,4.954383,52.394310,13300,2.0,Supermarket,Bus Stop,Fried Chicken Joint,Concert Hall,Grocery Store,Discount Store,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space
65,Tuindorp Oostzaan,4.893249,52.410703,11165,2.0,Bus Stop,Gym Pool,Theater,Snack Place,Chinese Restaurant,Business Service,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant


In [215]:
fourth=amsterdam_merged.loc[amsterdam_merged['Cluster Labels'] == 3]
fourth.sort_values(by="habitants",ascending=False)

,Neighborhood,lon,lat,habitants,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
98,Driemond,5.012156,52.313198,1475,3.0,Park,Zoo,Donut Shop,Dutch Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit


In [216]:
fifth=amsterdam_merged.loc[amsterdam_merged['Cluster Labels'] == 4]
fifth.sort_values(by="habitants",ascending=False)

,Neighborhood,lon,lat,habitants,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
57,Betondorp,4.940232,52.341272,3210,4.0,Bar,Bakery,Market,Zoo,Falafel Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit


## 4. Results section

We get 5 clusters of neighborhoods, based on their most common venues around, and the corresponding number of habitants. Now, let’s explore each cluster, to get most populated neighborhood candidates :

*1st Cluster* :  **Bijlmer Oost (E,G,K)**,  **Bijlmer Centrum (D,F,H)**, and **Jordaan**. This is Cluster has the most populated neighborhoods, and it’s  more likely to find clothes stores, Restaurants/Food places, Supermarkets,and transport stations. 

*2nd Cluster* : **Oostzanerwerf**. There are more public places like Soccer field, Event space and Exhebits

*3rd Cluster* : **Geuzenveld**, **Banne Buiksloot**. This cluster has an average populated neighborhoods, with most common venues like transport stations, supermarkets and restaurants

*4th Cluster* : **Driemond**. Is the least populated neighborhood, with public places ike park and zoo.

*5th Cluster* : **Betondorp**. Few population, and we can find places like Bars and Bekery.

## 5. Discussion section

According to profils that we get from results section, we can get some recommandations to start with a suitable business in each neighborhood. 

First, if we take the 1st Cluster, we can observe that there are the most populated and animated neighborhoods, but there aren’t some vital places like **Gym clubs** or **Fitness centers**. In the same way, the 3rd Cluster has populated neighborhoods, but there aren’t stores like **Clothes stores** or **Book stores**.

In other hands, the 2nd cluster may  hasn’t much populations, but has more public places like Park and Soccer filds, and then much more flows of people, and it may be interesting to get **groceries** or special stores like **Sport stores** or **Ethic stores**.  

## 6. Conclusion section:

We can say that our analysis of neighborhoods and their profil could help stakeholders to take decision about what kind of business to start with, and which neighborhoods is the most suitable. Note that our approach to solve this project take the number of population in neighborhoods out of clustering stage, and it serve to compare in the final stage between neighborhoods. The reason behind this aproach is to let stakeholders to decide the suitable business according to number of population.

Therefore, in this project we could also add other variables based on demographic informations that influence the choice of business nature, like gender or age.  